タスク
1. データは推移観測所の評価対象のもののみに絞る
2. 一部データの異常値修正(向原,今津,福田,津之郷,中地)の一部日付を削除
2. 一部データの異常値を修正する（★白川、大谷池に加え、山手左岸(国)、上安、七社、伊尾、中野を新たに修正、一部削除）
3. 以下のイメージで特徴量作成して結合

```
# イメージ
雨量(11)、時間(2)、stationは変えず。

潮位0
減らす
当日 h-23,22,21,18,15,12,6,0, min,max,mean,std,null_count
1日前 6時間ぶん平均、std,null_count
2日前 12時間分平均,std,null_count

53=> 37
-----
潮位1
当日24時間分、12時間分平均、min,max,mean,std,null_count
1日前 24時間分、12時間分平均min,max,mean,std,null_count
2日前 12時間分平均min,max,mean,std,null_count
3日前 12時間分平均min,max,mean,std,null_count
4日前 mean
5日前 mean
14日前 std
15日前 std
16日前 std

66=> 14+31*2+7*2+5=95
```

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
path = '../data/'

# # 雨量のデータの読み込み
# rain_data = pd.read_csv(path+'rainfall/data.csv')
# rain_stations = pd.read_csv(path+'rainfall/stations.csv')
# print('shapeの確認')
# print('雨量観測データ: ', rain_data.shape)
# print('雨量観測データ所: ', rain_stations.shape)

# # 潮位のデータの読み込み
# tide_data = pd.read_csv(path+'tidelevel/data.csv')
# tide_stations = pd.read_csv(path+'tidelevel/stations.csv')
# print('潮位観測データ: ', tide_data.shape)
# print('潮位観測データ所: ', tide_stations.shape)

# 水位のデータの読み込み
water_data = pd.read_csv(path+'waterlevel/data.csv')
water_stations = pd.read_csv(path+'waterlevel/stations.csv')
print('水位観測データ: ', water_data.shape)
print('水位観測データ所: ', water_stations.shape)

# # damデータの読み込み
# dam = pd.read_csv(path+'dam.csv')
# print('damデータ: ', dam.shape)

# 潮位ラベルや水位観測所に対応した雨量観測所の情報をまとめたもの
df_label = pd.read_excel('../data/process-data/雨量観測所対応and潮位ラベル.xlsx')

水位観測データ:  (392066, 27)
水位観測データ所:  (180, 12)


In [5]:
water_data.head()

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
0,0,釜ヶ原,小瀬川,0.76,0.76,0.76,0.76,0.76,0.76,0.76,...,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76,0.76
1,0,下ヶ原(国),小瀬川,0.71,0.71,0.71,0.71,0.71,0.71,0.71,...,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71
2,0,防鹿(国),小瀬川,4.71,4.71,4.7,4.7,4.71,4.71,4.71,...,4.7,4.7,4.71,4.7,4.7,4.7,4.71,4.7,4.7,4.7
3,0,本郷(廿日市),小瀬川,0.31,0.31,0.31,0.32,0.32,0.32,0.31,...,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32,0.32
4,0,駄荷(電),太田川,0.39,0.39,0.39,0.39,0.39,0.39,0.39,...,0.39,*,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4


In [6]:
water_stations.head()

,観測所名称,フリガナ,水系名,河川名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
0,多治比,タジヒ,江の川,多治比川,西部建設事務所,安芸高田市,河川課,安芸高田市吉田町多冶比,34.681944,132.667222,1,1
1,南原,ナバラ,太田川,南原川,西部建設事務所,広島市安佐北区,河川課,広島市安佐北区可部町上町屋字下小野1452番2,34.540556,132.501389,1,1
2,向原,ムカイハラ,太田川,三篠川,西部建設事務所,安芸高田市,河川課,安芸高田市向原町長田字田屋３４６２－２地先,34.607306,132.703333,1,1
3,上甲立,カミコウダチ,江の川,本村川,西部建設事務所,安芸高田市,河川課,安芸高田市甲田町上甲立字六地蔵１２５２－１地先,34.708333,132.739444,1,1
4,上安,カミヤス,太田川,安川,西部建設事務所,広島市安佐南区,河川課,広島市安佐南区相田２丁目地先,34.469306,132.460000,1,1


In [4]:
pd.set_option("display.max_columns", 30)

### 異常値修正

In [5]:
# 評価対象に絞る
L = list(water_stations[water_stations["評価対象"]==1]["観測所名称"])
water_data2 = water_data[water_data["station"].isin(L)]
water_data2_cp = water_data2.copy()
for i in range(24):
    water_data2_cp[f'{str(i).zfill(2)}:00:00'] = pd.to_numeric(water_data2_cp[f'{str(i).zfill(2)}:00:00'], errors='coerce')

In [6]:
# 向原
# date296~322を欠損に
loc = "向原"
start_date = 296
end_date = 322
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]==loc) & (water_data2_cp["date"]>=start_date) & (water_data2_cp["date"]<=end_date), f'{str(i).zfill(2)}:00:00'] = np.nan

In [7]:
# 今津
# date 1914~1941を欠損に
loc = "今津"
start_date = 1914
end_date = 1941
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]==loc) & (water_data2_cp["date"]>=start_date) & (water_data2_cp["date"]<=end_date), f'{str(i).zfill(2)}:00:00'] = np.nan

In [8]:
# 福田
# date1840~1900を欠損に
loc = "福田"
start_date = 1840
end_date = 1900
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]==loc) & (water_data2_cp["date"]>=start_date) & (water_data2_cp["date"]<=end_date), f'{str(i).zfill(2)}:00:00'] = np.nan

In [9]:
# 津之郷
# date810~825を欠損に
loc = "津之郷"
start_date = 810
end_date = 825
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]==loc) & (water_data2_cp["date"]>=start_date) & (water_data2_cp["date"]<=end_date), f'{str(i).zfill(2)}:00:00'] = np.nan

In [10]:
# 中地
# date275~308を欠損に
loc = "中地"
start_date = 275
end_date = 308
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]==loc) & (water_data2_cp["date"]>=start_date) & (water_data2_cp["date"]<=end_date), f'{str(i).zfill(2)}:00:00'] = np.nan

In [11]:
# 山手左岸(国)の値変更
for i in range(18,24):
    water_data2_cp.loc[(water_data2_cp["station"]=="山手左岸(国)") & (water_data2_cp["date"]==92), f'{str(i).zfill(2)}:00:00'] += 100
    
# 山手左岸(国)の値変更2
water_data2_cp.loc[(water_data2_cp["station"]=="山手左岸(国)") & (water_data2_cp["date"]==93), '00:00:00'] += 100

In [12]:
# 上安の異常値削る
water_data2_cp = water_data2_cp[~(water_data2_cp["station"]=="上安")|~(water_data2_cp["date"]>=300)|~(water_data2_cp["date"]<=440)]

In [13]:
# 七社の値変更
water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]==1356), '07:00:00'] *= -1
water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]==1356), '07:00:00'] += 10

for i in range(9,12):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1356), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1356), f'{str(i).zfill(2)}:00:00'] += 10
for i in range(13,24):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1356), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1356), f'{str(i).zfill(2)}:00:00'] += 10

for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]>= 1357) &(water_data2_cp["date"]<= 1358), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]>= 1357) &(water_data2_cp["date"]<= 1358), f'{str(i).zfill(2)}:00:00'] += 10

for i in range(16):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1359), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1359), f'{str(i).zfill(2)}:00:00'] += 10
for i in range(22,24):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1359), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1359), f'{str(i).zfill(2)}:00:00'] += 10

for i in range(6):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1360), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1360), f'{str(i).zfill(2)}:00:00'] += 10

for i in range(6,24):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1361), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1361), f'{str(i).zfill(2)}:00:00'] += 10    
    
for i in range(13):
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1362), f'{str(i).zfill(2)}:00:00'] *= -1
    water_data2_cp.loc[(water_data2_cp["station"]=="七社") & (water_data2_cp["date"]== 1362), f'{str(i).zfill(2)}:00:00'] += 10

In [14]:
# 伊尾,中野
water_data2_cp = water_data2_cp.sort_values(by=["date","00:00:00"])
water_data2_cp = water_data2_cp.drop_duplicates(subset=['date','station'], keep='first')

In [15]:
# 大谷池の値変更
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]=="大谷池") & (586<=water_data2_cp["date"]) & (water_data2_cp["date"]<=1302), f'{str(i).zfill(2)}:00:00'] += 100
    
# 大谷池の値変更その2
for i in range(13):
    water_data2_cp.loc[(water_data2_cp["station"]=="大谷池") & (water_data2_cp["date"]==1303), f'{str(i).zfill(2)}:00:00'] += 100

In [16]:
# 白川の値変更
for i in range(24):
    water_data2_cp.loc[(water_data2_cp["station"]=="白川") & (water_data2_cp["date"]<=697), f'{str(i).zfill(2)}:00:00'] -= 227.5
    
# 変更その2
for i in range(15):
    water_data2_cp.loc[(water_data2_cp["station"]=="白川") & (water_data2_cp["date"]==698), f'{str(i).zfill(2)}:00:00'] -= 227.5

### 前処理

In [21]:
# 潮位に似たデータだけにする
L_liketide = list(df_label[df_label["潮位ラベル"]==1]["水位観測所"])
water_data2_label1 = water_data2_cp[water_data2_cp["station"].isin(L_liketide)].copy()
water_data2_label1_cp = water_data2_label1.copy()

In [22]:
water_data2_label1_cp.head()

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
14,0,草津(国),太田川放水路,-0.26,-0.91,-1.50,-1.77,-1.50,-0.84,-0.08,0.60,1.31,1.83,1.91,1.56,0.96,0.27,-0.22,-0.65,-0.81,-0.44,0.17,0.74,1.17,1.48,1.43,0.86
15,0,祇園大橋(国),太田川,-0.17,-0.64,-0.69,-0.69,-0.68,-0.68,-0.25,0.54,1.24,1.85,1.95,1.55,1.03,0.33,-0.20,-0.54,-0.67,-0.56,0.15,0.68,1.14,1.47,1.48,0.93
11,0,江波(旧太田川)(国),旧太田川,-0.15,-0.79,-1.38,-1.71,-1.50,-0.86,-0.11,0.61,1.33,1.83,1.92,1.58,1.01,0.37,-0.14,-0.58,-0.78,-0.45,0.16,0.75,1.18,1.49,1.46,0.92
12,0,三篠橋(国),旧太田川,-0.13,-0.58,-0.91,-1.07,-1.11,-1.03,-0.32,0.52,1.21,1.84,1.94,1.58,1.03,0.33,-0.16,-0.50,-0.76,-0.58,0.08,0.68,1.12,1.46,1.46,0.95
17,0,長和久(国),太田川,0.15,0.12,0.13,0.14,0.14,0.15,0.15,0.49,1.11,1.77,1.87,1.59,1.11,0.49,0.16,0.14,0.16,0.24,0.26,0.64,1.07,1.41,1.45,1.05


In [26]:
water_data2_label1_cp[water_data2_label1_cp.columns[3:]].count(numeric_only=True,axis=1)[100:200]

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
2518,-0.27,-0.60,-0.82,-0.99,-1.07,-0.97,-0.38,0.29,0.82,1.31,1.45,1.14,0.68,0.12,-0.24,-0.51,-0.71,-0.56,-0.07,0.40,0.81,1.04,1.01,0.57
2517,-0.26,-0.76,-1.25,-1.52,-1.33,-0.80,-0.17,0.41,0.97,1.43,1.51,1.19,0.71,0.22,-0.17,-0.55,-0.68,-0.39,0.06,0.53,0.92,1.16,1.09,0.61
2544,0.25,0.25,0.25,0.24,0.24,0.24,0.25,0.36,0.92,1.39,1.45,1.12,0.64,0.26,0.25,0.25,NaN,0.25,0.25,0.48,0.89,1.14,1.02,0.55
2523,0.38,0.37,0.29,0.21,0.20,0.21,0.21,0.33,0.80,1.32,1.46,1.24,0.84,0.48,0.38,NaN,NaN,NaN,NaN,0.50,0.84,1.07,1.11,0.78
2592,0.67,0.46,0.44,0.43,0.43,0.43,0.43,0.44,1.13,1.69,1.96,1.82,1.48,1.10,0.62,0.45,0.44,0.44,0.43,0.53,1.08,1.52,1.60,1.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,0.44,0.44,0.44,0.44,0.44,0.44,1.01,1.56,1.82,1.80,1.54,1.19,0.74,0.47,0.44,0.44,0.44,0.51,0.99,1.40,1.56,1.40,1.04,0.60
5026,-1.00,-1.49,-1.72,-1.49,-0.90,-0.21,0.43,0.99,1.51,1.60,1.30,0.80,0.22,-0.18,-0.59,-0.77,-0.46,0.10,0.62,1.02,1.31,1.30,0.79,0.13
5023,-0.92,-1.41,-1.66,-1.45,-0.90,-0.21,0.43,1.01,1.52,1.62,1.34,0.85,0.29,-0.13,-0.52,-0.73,-0.46,0.10,0.64,1.04,1.33,1.34,0.83,0.20
5027,-0.68,-0.72,-0.72,-0.71,-0.71,-0.45,0.32,0.86,1.46,1.55,1.27,0.81,0.18,-0.23,-0.52,-0.68,-0.64,0.02,0.52,0.95,1.25,1.28,0.79,0.10


In [29]:
d = {}
for i in range(24):
    d[f'{str(i).zfill(2)}:00:00'] = f'h-{i}'
d

{'00:00:00': 'h-0',
 '01:00:00': 'h-1',
 '02:00:00': 'h-2',
 '03:00:00': 'h-3',
 '04:00:00': 'h-4',
 '05:00:00': 'h-5',
 '06:00:00': 'h-6',
 '07:00:00': 'h-7',
 '08:00:00': 'h-8',
 '09:00:00': 'h-9',
 '10:00:00': 'h-10',
 '11:00:00': 'h-11',
 '12:00:00': 'h-12',
 '13:00:00': 'h-13',
 '14:00:00': 'h-14',
 '15:00:00': 'h-15',
 '16:00:00': 'h-16',
 '17:00:00': 'h-17',
 '18:00:00': 'h-18',
 '19:00:00': 'h-19',
 '20:00:00': 'h-20',
 '21:00:00': 'h-21',
 '22:00:00': 'h-22',
 '23:00:00': 'h-23'}

In [30]:
# 潮位1加工
water_data2_label1['mean'] = water_data2_label1_cp[water_data2_label1_cp.columns[3:]].mean(axis=1)
water_data2_label1['max'] = water_data2_label1_cp[water_data2_label1_cp.columns[3:]].max(axis=1)
water_data2_label1['min'] = water_data2_label1_cp[water_data2_label1_cp.columns[3:]].min(axis=1)
water_data2_label1['std'] = water_data2_label1_cp[water_data2_label1_cp.columns[3:]].std(axis=1)
water_data2_label1['nullcount'] = water_data2_label1_cp[water_data2_label1_cp.columns[3:]].count(numeric_only=True,axis=1)
water_data2_label1["0-12mean"] = water_data2_label1_cp[water_data2_label1_cp.columns[3:15]].mean(axis=1)
water_data2_label1["12-24mean"] = water_data2_label1_cp[water_data2_label1_cp.columns[15:27]].mean(axis=1)

water_data2_label1 = water_data2_label1.rename(columns=d)

water_data2_label1

,date,station,river,h-0,h-1,h-2,h-3,h-4,h-5,h-6,h-7,h-8,h-9,h-10,h-11,...,h-16,h-17,h-18,h-19,h-20,h-21,h-22,h-23,mean,max,min,std,nullcount,0-12mean,12-24mean
14,0,草津(国),太田川放水路,-0.26,-0.91,-1.50,-1.77,-1.50,-0.84,-0.08,0.60,1.31,1.83,1.91,1.56,...,-0.81,-0.44,0.17,0.74,1.17,1.48,1.43,0.86,0.221250,1.91,-1.77,1.119159,24,0.029167,0.413333
15,0,祇園大橋(国),太田川,-0.17,-0.64,-0.69,-0.69,-0.68,-0.68,-0.25,0.54,1.24,1.85,1.95,1.55,...,-0.67,-0.56,0.15,0.68,1.14,1.47,1.48,0.93,0.357083,1.95,-0.69,0.933416,24,0.277500,0.436667
11,0,江波(旧太田川)(国),旧太田川,-0.15,-0.79,-1.38,-1.71,-1.50,-0.86,-0.11,0.61,1.33,1.83,1.92,1.58,...,-0.78,-0.45,0.16,0.75,1.18,1.49,1.46,0.92,0.256667,1.92,-1.71,1.104751,24,0.064167,0.449167
12,0,三篠橋(国),旧太田川,-0.13,-0.58,-0.91,-1.07,-1.11,-1.03,-0.32,0.52,1.21,1.84,1.94,1.58,...,-0.76,-0.58,0.08,0.68,1.12,1.46,1.46,0.95,0.293750,1.94,-1.11,1.006050,24,0.161667,0.425833
17,0,長和久(国),太田川,0.15,0.12,0.13,0.14,0.14,0.15,0.15,0.49,1.11,1.77,1.87,1.59,...,0.16,0.24,0.26,0.64,1.07,1.41,1.45,1.05,0.666250,1.87,0.12,0.609011,24,0.650833,0.681667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391931,2190,長和久(国),太田川,1.15,1.10,0.72,0.34,0.27,0.27,0.27,0.25,0.25,0.21,0.29,0.63,...,0.51,0.24,0.21,0.21,0.21,0.26,0.31,0.66,0.548750,1.46,0.21,0.407018,24,0.479167,0.618333
391935,2190,祇園大橋(国),太田川,1.16,0.96,0.49,0.03,-0.38,-0.60,-0.61,-0.62,-0.62,-0.33,0.23,0.57,...,0.32,0.05,-0.30,-0.59,-0.50,-0.07,0.21,0.61,0.189583,1.41,-0.62,0.671717,24,0.023333,0.355833
391964,2190,江波(旧太田川)(国),旧太田川,1.17,0.99,0.54,0.07,-0.40,-0.86,-1.19,-1.09,-0.66,-0.15,0.34,0.77,...,0.41,0.08,-0.30,-0.53,-0.37,-0.02,0.31,0.66,0.186667,1.46,-1.19,0.765044,24,-0.039167,0.412500
391936,2190,草津(国),太田川放水路,1.20,1.00,0.51,0.06,-0.43,-0.90,-1.19,-1.06,-0.64,-0.10,0.36,0.81,...,0.40,0.07,-0.32,-0.53,-0.34,0.01,0.35,0.68,0.196667,1.48,-1.19,0.772256,24,-0.031667,0.425000


In [31]:
# 1日前
water_data2_label1_1d_ago = water_data2_label1.copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_1d_ago.rename(columns={name: name+"_1d_ago"}, inplace=True)

water_data2_label1_1d_ago["date"] = water_data2_label1_1d_ago["date"] + 1
water_data2_label1_1d_ago.head()

,date,station,river,h-0_1d_ago,h-1_1d_ago,h-2_1d_ago,h-3_1d_ago,h-4_1d_ago,h-5_1d_ago,h-6_1d_ago,h-7_1d_ago,h-8_1d_ago,h-9_1d_ago,h-10_1d_ago,h-11_1d_ago,...,h-16_1d_ago,h-17_1d_ago,h-18_1d_ago,h-19_1d_ago,h-20_1d_ago,h-21_1d_ago,h-22_1d_ago,h-23_1d_ago,mean_1d_ago,max_1d_ago,min_1d_ago,std_1d_ago,nullcount_1d_ago,0-12mean_1d_ago,12-24mean_1d_ago
14,1,草津(国),太田川放水路,-0.26,-0.91,-1.50,-1.77,-1.50,-0.84,-0.08,0.60,1.31,1.83,1.91,1.56,...,-0.81,-0.44,0.17,0.74,1.17,1.48,1.43,0.86,0.221250,1.91,-1.77,1.119159,24,0.029167,0.413333
15,1,祇園大橋(国),太田川,-0.17,-0.64,-0.69,-0.69,-0.68,-0.68,-0.25,0.54,1.24,1.85,1.95,1.55,...,-0.67,-0.56,0.15,0.68,1.14,1.47,1.48,0.93,0.357083,1.95,-0.69,0.933416,24,0.277500,0.436667
11,1,江波(旧太田川)(国),旧太田川,-0.15,-0.79,-1.38,-1.71,-1.50,-0.86,-0.11,0.61,1.33,1.83,1.92,1.58,...,-0.78,-0.45,0.16,0.75,1.18,1.49,1.46,0.92,0.256667,1.92,-1.71,1.104751,24,0.064167,0.449167
12,1,三篠橋(国),旧太田川,-0.13,-0.58,-0.91,-1.07,-1.11,-1.03,-0.32,0.52,1.21,1.84,1.94,1.58,...,-0.76,-0.58,0.08,0.68,1.12,1.46,1.46,0.95,0.293750,1.94,-1.11,1.006050,24,0.161667,0.425833
17,1,長和久(国),太田川,0.15,0.12,0.13,0.14,0.14,0.15,0.15,0.49,1.11,1.77,1.87,1.59,...,0.16,0.24,0.26,0.64,1.07,1.41,1.45,1.05,0.666250,1.87,0.12,0.609011,24,0.650833,0.681667


In [34]:
# 2日前
water_data2_label1_2d_ago = water_data2_label1[["date", "station", "river", "mean", "max", "min",
                                                "std", "nullcount", "0-12mean", "12-24mean"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_2d_ago.rename(columns={name: name+"_2d_ago"}, inplace=True)

water_data2_label1_2d_ago["date"] = water_data2_label1_2d_ago["date"] + 2
water_data2_label1_2d_ago.head()

,date,station,river,mean_2d_ago,max_2d_ago,min_2d_ago,std_2d_ago,nullcount_2d_ago,0-12mean_2d_ago,12-24mean_2d_ago
14,2,草津(国),太田川放水路,0.221250,1.91,-1.77,1.119159,24,0.029167,0.413333
15,2,祇園大橋(国),太田川,0.357083,1.95,-0.69,0.933416,24,0.277500,0.436667
11,2,江波(旧太田川)(国),旧太田川,0.256667,1.92,-1.71,1.104751,24,0.064167,0.449167
12,2,三篠橋(国),旧太田川,0.293750,1.94,-1.11,1.006050,24,0.161667,0.425833
17,2,長和久(国),太田川,0.666250,1.87,0.12,0.609011,24,0.650833,0.681667


In [35]:
# 3日前
water_data2_label1_3d_ago = water_data2_label1[["date", "station", "river", "mean", "max", "min",
                                                "std", "nullcount", "0-12mean", "12-24mean"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_3d_ago.rename(columns={name: name+"_3d_ago"}, inplace=True)

water_data2_label1_3d_ago["date"] = water_data2_label1_3d_ago["date"] + 3
water_data2_label1_3d_ago.head()

,date,station,river,mean_3d_ago,max_3d_ago,min_3d_ago,std_3d_ago,nullcount_3d_ago,0-12mean_3d_ago,12-24mean_3d_ago
14,3,草津(国),太田川放水路,0.221250,1.91,-1.77,1.119159,24,0.029167,0.413333
15,3,祇園大橋(国),太田川,0.357083,1.95,-0.69,0.933416,24,0.277500,0.436667
11,3,江波(旧太田川)(国),旧太田川,0.256667,1.92,-1.71,1.104751,24,0.064167,0.449167
12,3,三篠橋(国),旧太田川,0.293750,1.94,-1.11,1.006050,24,0.161667,0.425833
17,3,長和久(国),太田川,0.666250,1.87,0.12,0.609011,24,0.650833,0.681667


In [36]:
# 4日前
water_data2_label1_4d_ago = water_data2_label1[["date", "station", "river", "mean", "nullcount"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_4d_ago.rename(columns={name: name+"_4d_ago"}, inplace=True)

water_data2_label1_4d_ago["date"] = water_data2_label1_4d_ago["date"] + 4

# 5日前
water_data2_label1_5d_ago = water_data2_label1[["date", "station", "river", "mean", "nullcount"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_5d_ago.rename(columns={name: name+"_5d_ago"}, inplace=True)

water_data2_label1_5d_ago["date"] = water_data2_label1_5d_ago["date"] + 5
water_data2_label1_5d_ago.head()

,date,station,river,mean_5d_ago,nullcount_5d_ago
14,5,草津(国),太田川放水路,0.221250,24
15,5,祇園大橋(国),太田川,0.357083,24
11,5,江波(旧太田川)(国),旧太田川,0.256667,24
12,5,三篠橋(国),旧太田川,0.293750,24
17,5,長和久(国),太田川,0.666250,24


In [37]:
# 14日前
water_data2_label1_14d_ago = water_data2_label1[["date", "station", "river", "std", "nullcount"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_14d_ago.rename(columns={name: name+"_14d_ago"}, inplace=True)

water_data2_label1_14d_ago["date"] = water_data2_label1_14d_ago["date"] + 14

# 15日前
water_data2_label1_15d_ago = water_data2_label1[["date", "station", "river", "std", "nullcount"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_15d_ago.rename(columns={name: name+"_15d_ago"}, inplace=True)

water_data2_label1_15d_ago["date"] = water_data2_label1_15d_ago["date"] + 15

# 16日前
water_data2_label1_16d_ago = water_data2_label1[["date", "station", "river", "std", "nullcount"]].copy()
for name in water_data2_label1.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label1_16d_ago.rename(columns={name: name+"_16d_ago"}, inplace=True)

water_data2_label1_16d_ago["date"] = water_data2_label1_16d_ago["date"] + 16

water_data2_label1_16d_ago.head()

,date,station,river,std_16d_ago,nullcount_16d_ago
14,16,草津(国),太田川放水路,1.119159,24
15,16,祇園大橋(国),太田川,0.933416,24
11,16,江波(旧太田川)(国),旧太田川,1.104751,24
12,16,三篠橋(国),旧太田川,1.006050,24
17,16,長和久(国),太田川,0.609011,24


In [38]:
df_water_1 = pd.merge(water_data2_label1, water_data2_label1_1d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_2d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_3d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_4d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_5d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_14d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_15d_ago, on=["date", "station", "river"], how="left")
df_water_1 = pd.merge(df_water_1, water_data2_label1_16d_ago, on=["date", "station", "river"], how="left")
df_water_1.head()

,date,station,river,h-0,h-1,h-2,h-3,h-4,h-5,h-6,h-7,h-8,h-9,h-10,h-11,...,min_3d_ago,std_3d_ago,nullcount_3d_ago,0-12mean_3d_ago,12-24mean_3d_ago,mean_4d_ago,nullcount_4d_ago,mean_5d_ago,nullcount_5d_ago,std_14d_ago,nullcount_14d_ago,std_15d_ago,nullcount_15d_ago,std_16d_ago,nullcount_16d_ago
0,0,草津(国),太田川放水路,-0.26,-0.91,-1.50,-1.77,-1.50,-0.84,-0.08,0.60,1.31,1.83,1.91,1.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,祇園大橋(国),太田川,-0.17,-0.64,-0.69,-0.69,-0.68,-0.68,-0.25,0.54,1.24,1.85,1.95,1.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,江波(旧太田川)(国),旧太田川,-0.15,-0.79,-1.38,-1.71,-1.50,-0.86,-0.11,0.61,1.33,1.83,1.92,1.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,三篠橋(国),旧太田川,-0.13,-0.58,-0.91,-1.07,-1.11,-1.03,-0.32,0.52,1.21,1.84,1.94,1.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,長和久(国),太田川,0.15,0.12,0.13,0.14,0.14,0.15,0.15,0.49,1.11,1.77,1.87,1.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_water_1.columns

Index(['date', 'station', 'river', 'h-0', 'h-1', 'h-2', 'h-3', 'h-4', 'h-5',
       'h-6', 'h-7', 'h-8', 'h-9', 'h-10', 'h-11', 'h-12', 'h-13', 'h-14',
       'h-15', 'h-16', 'h-17', 'h-18', 'h-19', 'h-20', 'h-21', 'h-22', 'h-23',
       'mean', 'max', 'min', 'std', 'nullcount', '0-12mean', '12-24mean',
       'h-0_1d_ago', 'h-1_1d_ago', 'h-2_1d_ago', 'h-3_1d_ago', 'h-4_1d_ago',
       'h-5_1d_ago', 'h-6_1d_ago', 'h-7_1d_ago', 'h-8_1d_ago', 'h-9_1d_ago',
       'h-10_1d_ago', 'h-11_1d_ago', 'h-12_1d_ago', 'h-13_1d_ago',
       'h-14_1d_ago', 'h-15_1d_ago', 'h-16_1d_ago', 'h-17_1d_ago',
       'h-18_1d_ago', 'h-19_1d_ago', 'h-20_1d_ago', 'h-21_1d_ago',
       'h-22_1d_ago', 'h-23_1d_ago', 'mean_1d_ago', 'max_1d_ago', 'min_1d_ago',
       'std_1d_ago', 'nullcount_1d_ago', '0-12mean_1d_ago', '12-24mean_1d_ago',
       'mean_2d_ago', 'max_2d_ago', 'min_2d_ago', 'std_2d_ago',
       'nullcount_2d_ago', '0-12mean_2d_ago', '12-24mean_2d_ago',
       'mean_3d_ago', 'max_3d_ago', 'min_3d_ago', 

In [40]:
df_water_1.to_csv("../data/process-data/water-process-label1-v1-修正済み3.csv", index = False)

In [42]:
L_notliketide = list(df_label[df_label["潮位ラベル"]==0]["水位観測所"])
water_data2_label0 = water_data2_cp[water_data2_cp["station"].isin(L_notliketide)].copy()
water_data2_label0_cp = water_data2_label0.copy()
water_data2_label0_cp.head()

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
58,0,加計(国),太田川,-2.00,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-2.00,-2.01,-2.00,-1.99,-1.99,-1.99,-1.98,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99
66,0,下土師(国),江の川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62
176,0,今津,本郷川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.63,-0.63,-0.63
75,0,樋ノ詰,黒瀬川,-0.57,-0.57,-0.57,-0.57,-0.57,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58
27,0,上原橋(国),根谷川,-0.51,-0.51,-0.51,-0.51,-0.50,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.50,-0.50,-0.50,-0.50


In [43]:
# 潮位1加工
water_data2_label0['mean'] = water_data2_label0_cp[water_data2_label0_cp.columns[3:]].mean(axis=1)
water_data2_label0['max'] = water_data2_label0_cp[water_data2_label0_cp.columns[3:]].max(axis=1)
water_data2_label0['min'] = water_data2_label0_cp[water_data2_label0_cp.columns[3:]].min(axis=1)
water_data2_label0['std'] = water_data2_label0_cp[water_data2_label0_cp.columns[3:]].std(axis=1)
water_data2_label0['nullcount'] = water_data2_label0_cp[water_data2_label0_cp.columns[3:]].count(numeric_only=True,axis=1)
water_data2_label0["0-12mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[3:15]].mean(axis=1)
water_data2_label0["12-24mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[15:27]].mean(axis=1)
water_data2_label0["0-6mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[3:9]].mean(axis=1)
water_data2_label0["6-12mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[9:15]].mean(axis=1)
water_data2_label0["12-18mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[15:21]].mean(axis=1)
water_data2_label0["18-24mean"] = water_data2_label0_cp[water_data2_label0_cp.columns[21:27]].mean(axis=1)

water_data2_label0 = water_data2_label0.rename(columns=d)
for i in range(24):
    if i not in [0,6,12,15,18,21,22,23]:
        water_data2_label0 = water_data2_label0.drop(columns=f'h-{i}')
water_data2_label0.head(2)

,date,station,river,h-0,h-6,h-12,h-15,h-18,h-21,h-22,h-23,mean,max,min,std,nullcount,0-12mean,12-24mean,0-6mean,6-12mean,12-18mean,18-24mean
58,0,加計(国),太田川,-2.00,-1.99,-2.01,-1.99,-1.99,-1.99,-1.99,-1.99,-1.991667,-1.98,-2.01,0.005647,24,-1.991667,-1.991667,-1.991667,-1.991667,-1.993333,-1.99
66,0,下土師(国),江の川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.620000,-0.62,-0.62,0.000000,24,-0.620000,-0.620000,-0.620000,-0.620000,-0.620000,-0.62


In [44]:
# 1日前
water_data2_label0_1d_ago = water_data2_label0[["date", "station", "river", "std", "nullcount", "0-6mean", "6-12mean", "12-18mean", "18-24mean"]].copy()
for name in water_data2_label0.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label0_1d_ago.rename(columns={name: name+"_1d_ago"}, inplace=True)

water_data2_label0_1d_ago["date"] = water_data2_label0_1d_ago["date"] + 1

# 2日前
water_data2_label0_2d_ago = water_data2_label0[["date", "station", "river", "std", "nullcount", "0-12mean", "12-24mean"]].copy()
for name in water_data2_label0.columns:
    if name not in ["date", "station", "river"]:
        water_data2_label0_2d_ago.rename(columns={name: name+"_2d_ago"}, inplace=True)

water_data2_label0_2d_ago["date"] = water_data2_label0_2d_ago["date"] + 2
water_data2_label0_2d_ago.head()

,date,station,river,std_2d_ago,nullcount_2d_ago,0-12mean_2d_ago,12-24mean_2d_ago
58,2,加計(国),太田川,0.005647,24,-1.991667,-1.991667
66,2,下土師(国),江の川,0.000000,24,-0.620000,-0.620000
176,2,今津,本郷川,0.003378,24,-0.620000,-0.622500
75,2,樋ノ詰,黒瀬川,0.004149,24,-0.575833,-0.580000
27,2,上原橋(国),根谷川,0.004149,24,-0.509167,-0.506667


In [45]:
df_water_0 = pd.merge(water_data2_label0, water_data2_label0_1d_ago, on=["date", "station", "river"], how="left")
df_water_0 = pd.merge(df_water_0, water_data2_label0_2d_ago, on=["date", "station", "river"], how="left")
df_water_0.head()

,date,station,river,h-0,h-6,h-12,h-15,h-18,h-21,h-22,h-23,mean,max,min,std,...,12-24mean,0-6mean,6-12mean,12-18mean,18-24mean,std_1d_ago,nullcount_1d_ago,0-6mean_1d_ago,6-12mean_1d_ago,12-18mean_1d_ago,18-24mean_1d_ago,std_2d_ago,nullcount_2d_ago,0-12mean_2d_ago,12-24mean_2d_ago
0,0,加計(国),太田川,-2.00,-1.99,-2.01,-1.99,-1.99,-1.99,-1.99,-1.99,-1.991667,-1.98,-2.01,0.005647,...,-1.991667,-1.991667,-1.991667,-1.993333,-1.990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,下土師(国),江の川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.620000,-0.62,-0.62,0.000000,...,-0.620000,-0.620000,-0.620000,-0.620000,-0.620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,今津,本郷川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.63,-0.63,-0.63,-0.621250,-0.62,-0.63,0.003378,...,-0.622500,-0.620000,-0.620000,-0.620000,-0.625000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,樋ノ詰,黒瀬川,-0.57,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.577917,-0.57,-0.58,0.004149,...,-0.580000,-0.571667,-0.580000,-0.580000,-0.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,上原橋(国),根谷川,-0.51,-0.51,-0.51,-0.51,-0.51,-0.50,-0.50,-0.50,-0.507917,-0.50,-0.51,0.004149,...,-0.506667,-0.508333,-0.510000,-0.510000,-0.503333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_water_0.columns

Index(['date', 'station', 'river', 'h-0', 'h-6', 'h-12', 'h-15', 'h-18',
       'h-21', 'h-22', 'h-23', 'mean', 'max', 'min', 'std', 'nullcount',
       '0-12mean', '12-24mean', '0-6mean', '6-12mean', '12-18mean',
       '18-24mean', 'std_1d_ago', 'nullcount_1d_ago', '0-6mean_1d_ago',
       '6-12mean_1d_ago', '12-18mean_1d_ago', '18-24mean_1d_ago', 'std_2d_ago',
       'nullcount_2d_ago', '0-12mean_2d_ago', '12-24mean_2d_ago'],
      dtype='object')

In [47]:
df_water_0.to_csv("../data/process-data/water-process-label0-v1-修正済み3.csv", index = False)

In [46]:
# train4.csvも値変更したいので別のDataFrame作成
water_data2_cp

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,10:00:00,11:00:00,12:00:00,13:00:00,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
58,0,加計(国),太田川,-2.00,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99,-2.00,-2.01,-2.00,-1.99,-1.99,-1.99,-1.98,-1.99,-1.99,-1.99,-1.99,-1.99,-1.99
66,0,下土師(国),江の川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62
176,0,今津,本郷川,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.62,-0.63,-0.63,-0.63
75,0,樋ノ詰,黒瀬川,-0.57,-0.57,-0.57,-0.57,-0.57,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58,-0.58
27,0,上原橋(国),根谷川,-0.51,-0.51,-0.51,-0.51,-0.50,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.51,-0.50,-0.50,-0.50,-0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391942,2190,防鹿(国),小瀬川,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68,4.68
391920,2190,大谷池,四川,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23,302.23
391891,2190,上庄(国),三篠川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392021,2190,駅前,芦田川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# 1時間ごとのデータに
df_water_per_hour = pd.DataFrame(water_data2_cp.set_index(["date", "station", "river"]).stack(dropna=False)).reset_index().rename(columns={0:"water level", "level_3":"hour"})

In [48]:
df_water_per_hour["hour"] = df_water_per_hour["hour"].str[:2].astype(int)
df_water_per_hour["hour_sin"] = np.sin(np.pi*df_water_per_hour["hour"]/12)
df_water_per_hour["hour_cos"] = np.cos(np.pi*df_water_per_hour["hour"]/12)

In [49]:
df_water_per_hour.head()

,date,station,river,hour,water level,hour_sin,hour_cos
0,0,加計(国),太田川,0,-2.00,0.000000,1.000000
1,0,加計(国),太田川,1,-1.99,0.258819,0.965926
2,0,加計(国),太田川,2,-1.99,0.500000,0.866025
3,0,加計(国),太田川,3,-1.99,0.707107,0.707107
4,0,加計(国),太田川,4,-1.99,0.866025,0.500000


In [50]:
df_water_per_hour[(df_water_per_hour["station"]=="大谷池") & (df_water_per_hour["date"]>= 571) &(df_water_per_hour["date"]<= 590)]

,date,station,river,hour,water level,hour_sin,hour_cos
2275440,571,大谷池,四川,0,305.14,0.000000,1.000000
2275441,571,大谷池,四川,1,305.14,0.258819,0.965926
2275442,571,大谷池,四川,2,305.14,0.500000,0.866025
2275443,571,大谷池,四川,3,305.14,0.707107,0.707107
2275444,571,大谷池,四川,4,305.14,0.866025,0.500000
...,...,...,...,...,...,...,...
2350651,590,大谷池,四川,19,NaN,-0.965926,0.258819
2350652,590,大谷池,四川,20,NaN,-0.866025,0.500000
2350653,590,大谷池,四川,21,NaN,-0.707107,0.707107
2350654,590,大谷池,四川,22,NaN,-0.500000,0.866025


In [51]:
df_a = df_water_per_hour[["date", "hour", "station", "water level"]].copy()
df_a["date"] = df_a["date"] - 1
df_a = df_a.rename(columns={'water level': 'next water level'})
df_a.head()

,date,hour,station,next water level
0,-1,0,加計(国),-2.00
1,-1,1,加計(国),-1.99
2,-1,2,加計(国),-1.99
3,-1,3,加計(国),-1.99
4,-1,4,加計(国),-1.99


In [52]:
df = pd.merge(df_water_per_hour, df_a, on=["date", "hour", "station"], how="left")

In [53]:
df.shape

(8698848, 8)

In [54]:
df = df.dropna(subset=["next water level"])

In [55]:
df.shape

(8173420, 8)

In [56]:
df.head()

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
0,0,加計(国),太田川,0,-2.00,0.000000,1.000000,-1.99
1,0,加計(国),太田川,1,-1.99,0.258819,0.965926,-1.99
2,0,加計(国),太田川,2,-1.99,0.500000,0.866025,-1.99
3,0,加計(国),太田川,3,-1.99,0.707107,0.707107,-1.99
4,0,加計(国),太田川,4,-1.99,0.866025,0.500000,-1.99


In [57]:
df[(df["station"]=="大谷池") & (df["date"]>= 570) &(df["date"]<= 590)]

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
2270952,570,大谷池,四川,0,NaN,0.000000,1.000000,305.14
2270953,570,大谷池,四川,1,NaN,0.258819,0.965926,305.14
2270954,570,大谷池,四川,2,NaN,0.500000,0.866025,305.14
2270955,570,大谷池,四川,3,NaN,0.707107,0.707107,305.14
2270956,570,大谷池,四川,4,NaN,0.866025,0.500000,305.14
...,...,...,...,...,...,...,...,...
2330729,585,大谷池,四川,17,NaN,-0.965926,-0.258819,305.14
2330731,585,大谷池,四川,19,NaN,-0.965926,0.258819,305.14
2330733,585,大谷池,四川,21,NaN,-0.707107,0.707107,305.14
2330734,585,大谷池,四川,22,NaN,-0.500000,0.866025,305.14


In [58]:
df[(df["station"]=="大谷池") & (df["date"]== 585)]

,date,station,river,hour,water level,hour_sin,hour_cos,next water level
2330712,585,大谷池,四川,0,NaN,0.000000e+00,1.000000e+00,305.14
2330713,585,大谷池,四川,1,NaN,2.588190e-01,9.659258e-01,305.14
2330714,585,大谷池,四川,2,NaN,5.000000e-01,8.660254e-01,305.14
2330715,585,大谷池,四川,3,NaN,7.071068e-01,7.071068e-01,305.14
2330716,585,大谷池,四川,4,NaN,8.660254e-01,5.000000e-01,305.14
2330717,585,大谷池,四川,5,NaN,9.659258e-01,2.588190e-01,305.14
2330718,585,大谷池,四川,6,NaN,1.000000e+00,6.123234e-17,305.14
2330719,585,大谷池,四川,7,NaN,9.659258e-01,-2.588190e-01,305.14
2330720,585,大谷池,四川,8,NaN,8.660254e-01,-5.000000e-01,305.14
2330722,585,大谷池,四川,10,NaN,5.000000e-01,-8.660254e-01,305.14


In [59]:
df.to_csv("../data/process-data/train4-修正済み3.csv", index = False)